# Fronius Solar API

In [133]:
import requests
import user_data as ud
import pandas as pd

url_base = 'http://{}/solar_api'.format(ud.ip)
url = url_base + '/GetAPIVersion.cgi'

pd.DataFrame(requests.get(url).json(), [0]).set_index('APIVersion')

,BaseURL,CompatibilityRange
APIVersion,,
1,/solar_api/v1/,1.5-18


## Real-Time Data
- DAY_ENERGY: AC Energy generated on current day
- PAC: AC power (negative value for consuming power)
- TOTAL_ENERGY: AC Energy generated overall
- YEAR_ENERGY: AC Energy generated in current year

### Scope = System

In [134]:
url_real = url_base + '/v1/GetInverterRealtimeData.cgi'
url = url_real + '?Scope=System'

dic = requests.get(url).json()
dic['Head']

{'RequestArguments': {'DeviceClass': 'Inverter', 'Scope': 'System'},
 'Status': {'Code': 0, 'Reason': '', 'UserMessage': ''},
 'Timestamp': '2021-03-12T17:46:14+01:00'}

In [135]:
df = pd.DataFrame(dic['Body']['Data'])
df.loc['Values'] = df.loc['Values'].apply(lambda x: x['1'])
df

,DAY_ENERGY,PAC,TOTAL_ENERGY,YEAR_ENERGY
Unit,Wh,W,Wh,Wh
Values,8853,27,4542970,551051


### Scope = Device
#### Cumulation Inverter Data
Data is identical to "System".

In [136]:
url_real_dev = url_real + '?Scope=Device&DeviceId=1'
url = url_real_dev + '&DataCollection=CumulationInverterData'

dic = requests.get(url).json()
dic_data = dic['Body']['Data']
dic['Head'], dic_data['DeviceStatus']

({'RequestArguments': {'DataCollection': 'CumulationInverterData',
   'DeviceClass': 'Inverter',
   'DeviceId': '1',
   'Scope': 'Device'},
  'Status': {'Code': 0, 'Reason': '', 'UserMessage': ''},
  'Timestamp': '2021-03-12T17:46:15+01:00'},
 {'ErrorCode': 0,
  'LEDColor': 2,
  'LEDState': 0,
  'MgmtTimerRemainingTime': -1,
  'StateToReset': False,
  'StatusCode': 7})

In [137]:
df = {key: dic_data[key] for key in dic_data.keys() if key != 'DeviceStatus'}
pd.DataFrame(df)

,DAY_ENERGY,PAC,TOTAL_ENERGY,YEAR_ENERGY
Unit,Wh,W,Wh,Wh
Value,8853,20,4542970,551051


#### Common Inverter Data
- FAC: AC frequency
- IAC: AC current (absolute, accumulated over all lines)
- IDC: DC current
- UAC: AC voltage
- UDC: DC voltage

In [138]:
url = url_real_dev + '&DataCollection=CommonInverterData'

dic = requests.get(url).json()
dic_data = dic['Body']['Data']
dic['Head'], dic_data['DeviceStatus']

({'RequestArguments': {'DataCollection': 'CommonInverterData',
   'DeviceClass': 'Inverter',
   'DeviceId': '1',
   'Scope': 'Device'},
  'Status': {'Code': 0, 'Reason': '', 'UserMessage': ''},
  'Timestamp': '2021-03-12T17:46:16+01:00'},
 {'ErrorCode': 0,
  'LEDColor': 2,
  'LEDState': 0,
  'MgmtTimerRemainingTime': -1,
  'StateToReset': False,
  'StatusCode': 7})

In [139]:
df = {key: dic_data[key] for key in dic_data.keys() if key != 'DeviceStatus'}
pd.DataFrame(df)

,DAY_ENERGY,FAC,IAC,IDC,PAC,TOTAL_ENERGY,UAC,UDC,YEAR_ENERGY
Unit,Wh,Hz,A,A,W,Wh,V,V,Wh
Value,8853,49.97,0.12,1.01,19,4542970,233.1,71.5,551051


#### Min Max Inverter Data

In [140]:
url = url_real_dev + '&DataCollection=MinMaxInverterData'

dic = requests.get(url).json()
dic['Head']

{'RequestArguments': {'DataCollection': 'MinMaxInverterData',
  'DeviceClass': 'Inverter',
  'DeviceId': '1',
  'Scope': 'Device'},
 'Status': {'Code': 0, 'Reason': '', 'UserMessage': ''},
 'Timestamp': '2021-03-12T17:46:17+01:00'}

In [141]:
pd.DataFrame(dic['Body']['Data'])

,DAY_PMAX,DAY_UACMAX,DAY_UDCMAX,TOTAL_PMAX,TOTAL_UACMAX,TOTAL_UDCMAX,YEAR_PMAX,YEAR_UACMAX,YEAR_UDCMAX
Unit,W,V,V,W,V,V,W,V,V
Value,5077,242.6,219.6,5216,267,246.1,5077,267,242.7


### Get Logger Info
Information about logging device.

In [142]:
url = url_base + '/v1/GetLoggerInfo.cgi'
# requests.get(url).json()

### Get Logger LED Info
Information of LEDs on the device.

In [143]:
url = url_base + '/v1/GetLoggerLEDInfo.cgi'
dic = requests.get(url).json()
dic['Head']

{'RequestArguments': {},
 'Status': {'Code': 0, 'Reason': '', 'UserMessage': ''},
 'Timestamp': '2021-03-12T17:46:19+01:00'}

In [144]:
pd.DataFrame(dic['Body']['Data'])

,PowerLED,SolarNetLED,SolarWebLED,WLANLED
Color,green,green,green,green
State,on,on,on,on


### Get Inverter Info
Information about all inverters who were online in the last 24 hours.

In [145]:
url = url_base + '/v1/GetInverterInfo.cgi'
# requests.get(url).json()

### Get Active Device Info
Information about online decives.

In [146]:
url = url_base + '/v1/GetActiveDeviceInfo.cgi?DeviceClass=System'
# requests.get(url).json()

### Get Power Flow Realtime Data
Information about the local energy grid.

In [147]:
url = url_base + '/v1/GetPowerFlowRealtimeData.fcgi'
dic = requests.get(url).json()
dic_dat = dic['Body']['Data']
dic['Head'] # , {key: dic_dat[key] for key in dic_dat.keys() if key != 'Inverters'}

{'RequestArguments': {},
 'Status': {'Code': 0, 'Reason': '', 'UserMessage': ''},
 'Timestamp': '2021-03-12T17:46:19+01:00'}

In [148]:
pd.DataFrame(dic_dat['Inverters'])

,1
DT,76
E_Day,8853
E_Total,4542970
E_Year,551051
P,33
